# Llama 3.2 1B 모델 파인튜닝 
- 타겟 모델 : `llama-3-2-1b 모델`
- 참조 github : [참조](https://github.com/aws/amazon-sagemaker-examples/blob/default/%20%20%20%20generative_ai/sm-jumpstart_foundation_llama_3_2_3b_finetuning.ipynb)


In [ ]:
model_id, model_version = "meta-textgeneration-llama-3-2-1b", "1.*"

## 샘플 데이터셋 준비
허깅페이스에 공개된 dolly 데이터셋을 다운로드 한다. dolly 데이터셋의 전체는 수십 Gb를 넘는 대량이므로 15k로 만들어 놓은 subset 데이터를 사용해 본다. 


In [14]:
local_data_file = "train.jsonl"

In [15]:
from datasets import load_dataset

# 데이터셋 로드 
dolly_dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

# summarization 카테고리 데이터만 추출한다.
summarization_dataset = dolly_dataset.filter(lambda example: example["category"] == "summarization")
summarization_dataset = summarization_dataset.remove_columns("category")

# 훈련/테스트 데이터셋으로 분리 한다. 
train_and_test_dataset = summarization_dataset.train_test_split(test_size=0.1)

# jsonl 파일로 변경, 저장한다. jsonl은 한 행이 하나의 row 이다. "json"과 달리 행을 변경할 때 ","가 없다. 
train_and_test_dataset["train"].to_json(local_data_file)

# 데이터셋이 가지고 있는 column을 살펴 본다. 
print(train_and_test_dataset["train"][0].keys())

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

dict_keys(['instruction', 'context', 'response'])


## 데이터 템플릿 
- 결국 Model에 입력해주는 형식은 `prompt`와 `complete` 형식으로 질문과 정답을 넣어준다. (SFT 학습)
- 이중, llama는 `prompt`의 특정 현태를 따르도록 강제하고 있으므로 그 형식에 맞게 가공해 주어야 한다.
- 따라서 템플릿은 `prompt` 문자열을 다음과 같은 형태로 만들어 주기 위함이다. 
```josn
{
    "prompt":"시스템 메세지
    
    ### Instruction:
    {인스트럭션 문자열}
    
    ###Input:
    {질문 문자열}
    ",
    "completeion":"{대답 문자열}"
}
```

이러한 형태를 만들기 위해서 별도의 템플릿 파일 `template.json`을 동일 폴더에 업로드하고, 폴더(버킷)위치를 입력으로 주면 `template.json`을 자동으로 검색하여 적용한다. 
이때, `train.jsonl`의 컬럼이름과 template의 포맷변수를 정확하게 매칭 시켜 주어야 한다. 가령, `train.jsonl`의 컬럼이름이 `instruction`, `context`, `response`라면, 템플릿에서도 동일한 명칭으로 포맷팅 되어 있어야 한다.

In [4]:
import json

template = {
    "prompt": "Below is an instruction that describes a task, paired with an input that provides further context. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{instruction}\n\n### Input:\n{context}\n\n",
    "completion": " {response}",
}
with open("template.json", "w") as f:
    json.dump(template, f)

## 데이터 버킷 업로드
- 모델을 훈련할 때 사용할 데이터를 업로드 한다.
- Sagemaker Training Job은 결국 Docker/Container 이므로, Container가 실행되는 환경에 데이터를 삽입해 주어야 한다.
- Container를 시작할 때 데이터를 직접 주입해주는 방법도 있으나, 시간/리소스/표준형태여부 등으로 잘 사용하지 않고,
- 일반적으로 S3 버켓에 업로드하고 해당 버킷주소를 파라미터로 넘겨주는 방식을 사용한다. fit(`{"training": <<s3버킷주소>> }`)
- Sagemaker는 컨테이너를 실행 할 때 버킷에서 데이터를 다운로드하여 data 폴더에 복사해 둔다.


> [참조] 이러한 데이터를 관련하는 사전정의된 폴더들을 별도 관리한다.
- /opt/ml/input/data/{channel_name}: 사용자 정의 채널
- /opt/ml/output: 훈련 출력 저장 위치

> 이번 예제의 경우, `/opt/ml/input/data/training/` 에 데이터를 복사해 두고, 내부적으로 훈련데이터를 접근할 때 해당 폴더를 참조하게 된다.


### [참고] 훈련 환경 (Training Environment)

1. 입력 데이터 관련:
- /opt/ml/input/data/training: 기본 훈련 데이터 채널
- /opt/ml/input/data/validation: 검증 데이터 채널 (사용 시)
- /opt/ml/input/data/test: 테스트 데이터 채널 (사용 시)
- /opt/ml/input/data/{channel_name}: 사용자 정의 채널

2. 설정 관련:
- /opt/ml/input/config/hyperparameters.json: 하이퍼파라미터 설정
- /opt/ml/input/config/resourceconfig.json: 리소스 구성 정보
- /opt/ml/input/config/trainingjobconfig.json: 훈련 작업 구성 정보

3. 출력 관련:
- /opt/ml/model: 훈련된 모델 저장 위치 (S3에 업로드됨)
- /opt/ml/output: 훈련 출력 저장 위치
- /opt/ml/output/data: 추가 출력 데이터 저장 위치
- /opt/ml/output/failure: 실패 정보 저장 위치

4. 체크포인트 관련:
- /opt/ml/checkpoints: 모델 체크포인트 저장 위치

In [7]:
bucket_name = "my-model-train"
bucket_prefix = "finetune"

In [10]:
from sagemaker.s3 import S3Uploader
import sagemaker
import random


train_data_location = f"s3://{bucket_name}/{bucket_prefix}/dolly_dataset"
S3Uploader.upload(local_data_file, train_data_location)
S3Uploader.upload("template.json", train_data_location)
print(f"Training data: {train_data_location}")

Training data: s3://my-model-train/finetune/dolly_dataset


# 학습 (파인튜닝)
Sagemaker / JumpStartEstimator를 이용한 학습 실행
- model_id와 버전을 입력합니다.
- `environment={"accept_eula": "true"}`로 EULA 라이선스를 동의 해 주어야 합니다.

estimator 생성이 끝나면, `set_hyperparameters`를 통하여 모델 파라미터 세팅해 주고, `fit()`함수를 실행 해 줍니다. 
- fit()함수를 실행 할 때, `channel_name`으로 `training`에 위에 업로드한 S3버킷주소(폴더)를 입력해 줍니다. (train.jsonl 뿐만아니라 template.json이 포함되어 있으므로 폴더 단위로 입력 해야 합니다.)

In [13]:
from sagemaker.jumpstart.estimator import JumpStartEstimator


estimator = JumpStartEstimator(
    model_id=model_id,
    model_version=model_version,
    environment={"accept_eula": "true"},  # EULA 동의
    disable_output_compression=True,
)

estimator.set_hyperparameters(instruction_tuned="True", epoch="1", max_input_length="1024")
estimator.fit({"training": train_data_location})

Using model 'meta-textgeneration-llama-3-2-1b' with wildcard version identifier '1.*'. You can pin to version '1.2.1' for more stable results. Note that models may have different input/output signatures after a major version upgrade.
INFO:sagemaker:Creating training-job with name: meta-textgeneration-llama-3-2-1b-2025-04-13-09-33-32-320


2025-04-13 09:33:33 Starting - Starting the training job
2025-04-13 09:33:33 Pending - Training job waiting for capacity...
2025-04-13 09:34:03 Pending - Preparing the instances for training...
2025-04-13 09:34:30 Downloading - Downloading input data...............
2025-04-13 09:36:46 Downloading - Downloading the training image............
2025-04-13 09:38:42 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2025-04-13 09:39:06,247 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-04-13 09:39:06,265 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-04-13 09:39:06,274 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2025-04-13 09:39:06,276 sagemaker_pytorch_container.training INFO     Invoking user training script.
2025-04-13 09:3

# Deployment 
파인튜닝한 모델의 결과물은, sagemaker container 내부 `/opt/ml/model`에 저장되는데 훈련이 종료되는 시점에 자동으로 S3에 업로드 됩니다. 업로드 S3 위치는 (특별히 지정하지 않는다면) Sagemaker의 기본 버킷하위에 `{job_name}/output/model/`위치에 저장됩니다. 
- 해당 bucket의 위치를 model 파일로 하여 deploy 할 수 도 있고,
- 아래처럼 훈련 제어 변수인 `estimator.deploy()`를 통해서 배포할 수 도 있습니다. 

In [17]:
finetuned_predictor = estimator.deploy(endpoint_name="03-llama-basic") # 엔트포인트 이름에 언더바`_` 불가

No instance type selected for inference hosting endpoint. Defaulting to ml.g6.xlarge.
INFO:sagemaker.jumpstart:No instance type selected for inference hosting endpoint. Defaulting to ml.g6.xlarge.
INFO:sagemaker:Creating model with name: meta-textgeneration-llama-3-2-1b-2025-04-13-12-13-02-516
INFO:sagemaker:Creating endpoint-config with name 03-llama-basic
INFO:sagemaker:Creating endpoint with name 03-llama-basic


------------!

In [59]:

template_inference = template['prompt']

instruction = "너의 이름은 무엇이야?"
input_output_marker = "\n\n### Response:\n"
prompt = template["prompt"].format(instruction=instruction, context=context)
prompt += input_output_marker

prompt_payload = {
    'inputs':prompt,
    'parameters': {'max_new_tokens': 512, 'top_p': 0.9,'temperature': 0.6}
    # 'parameters': {'max_new_tokens': 512, 'top_p': 0.9,'temperature': 0.6, 'details': True}
}

In [60]:
response = finetuned_predictor.predict(prompt_payload)

In [61]:
response['generated_text']

' 이상재는 대한민국의 전직 축구 선수이자 현 축구 감독이다.'

# Inference with sagemaker runtime

In [58]:
import boto3

# SageMaker Runtime 클라이언트 생성
sagemaker_runtime = boto3.client('sagemaker-runtime', region_name='us-west-2')

# 엔드포인트 이름
endpoint_name = "03-llama-basic"


# JSON 형식으로 데이터 직렬화
payload = json.dumps(prompt_payload)

# 엔드포인트 호출
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,  # 엔드포인트 이름
    Body=payload,               # 입력 데이터
    ContentType='application/json'  # 데이터 형식
)

# 응답 처리
result = json.loads(response['Body'].read().decode('utf-8'))
print("Model Response:\n", result['generated_text'].strip())


Model Response:
 이상재는 대한민국의 전직 축구 선수이자 현 축구 감독이다.
